In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
response= requests.get('https://www.imdb.com/search/title/?title_type=feature&num_votes=5000,&languages=en&sort=boxoffice_gross_us,desc&explore=genres&view=advanced')
soup = BeautifulSoup(response.content, "html.parser")

# Find the table element you want to scrape
movies_divs = soup.find_all("div", class_="lister-item-content")

movies=[]
for movie in movies_divs:
    title=movie.find('a').text
    year=movie.find('span', class_='lister-item-year').text.strip('()')
    runtime=movie.find('span', class_='runtime').text.strip(' min')
    genre=movie.find('span', class_='genre').text.strip()
    imdb=movie.find('strong').text
    metascore=movie.find('span', class_='metascore').text.strip()
    description=movie.find('p').find_next_sibling('p').text
    description=description.replace('\n','')
    director_star_element=movie.find('p', class_="").text
    split=director_star_element.split('|')
    director_part = split[0]
    stars_part = split[1]
    director = director_part.replace("Director:", "")
    director = director.replace("Directors:", "")
    director = director.replace("\n", "")
    stars = stars_part.replace("\n", "")
    stars = stars.replace(" Stars:", "").split(", ")
    votes_element = movie.find("span", attrs={"name": "nv", "data-value": True})
    votes = votes_element["data-value"] if votes_element else None
    gross_element = votes_element.find_next_sibling("span", attrs={"name": "nv", "data-value": True})
    gross = gross_element["data-value"] if gross_element else None

    movie_data = {
        "Title": title,
        "Year": year,
        "Runtime (Minutes)":runtime,
        "Genre": genre,
        "IMDB": imdb,
        "Metascore": metascore,
        "Description":description,
        "Director": director.strip(),
        "Stars": ','.join(stars).strip(),
        "Votes": votes,
        "Gross ($)": gross
    }
    movies.append(movie_data)
    

In [5]:
x=51
for i in range(0,246):
    url='https://www.imdb.com/search/title/?title_type=feature&num_votes=5000,&languages=en&sort=boxoffice_gross_us,desc&start='+str(x)+'&explore=genres&ref_=adv_nxt'
    response= requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table element you want to scrape
    movies_divs = soup.find_all("div", class_="lister-item-content")

    for movie in movies_divs:
        title=movie.find('a').text
        year=movie.find('span', class_='lister-item-year').text.strip('()')
        runtime=movie.find('span', class_='runtime').text.strip(' min')
        genre=movie.find('span', class_='genre').text.strip()
        imdb=movie.find('strong').text
        metascore_element=movie.find('span', class_='metascore')
        metascore=metascore_element.text.strip() if metascore_element else None
        description=movie.find('p').find_next_sibling('p').text
        description=description.replace('\n','')
        director_star_element=movie.find('p', class_="").text
        split=director_star_element.split('|')
        director_part = split[0]
        stars_part = split[1]
        director = director_part.replace("Director:", "")
        director = director.replace("Directors:", "")
        director = director.replace("\n", "")
        stars = stars_part.replace("\n", "")
        stars = stars.replace(" Stars:", "").split(", ")
        votes_element = movie.find("span", attrs={"name": "nv", "data-value": True})
        votes = votes_element["data-value"] if votes_element else None
        gross_element = votes_element.find_next_sibling("span", attrs={"name": "nv", "data-value": True})
        gross = gross_element["data-value"] if gross_element else None

        movie_data = {
            "Title": title,
            "Year": year,
            "Runtime (Minutes)":runtime,
            "Genre": genre,
            "IMDB": imdb,
            "Metascore": metascore,
            "Description":description,
            "Director": director.strip(),
            "Stars": ','.join(stars).strip(),
            "Votes": votes,
            "Gross ($)": gross
        }
        movies.append(movie_data)
    x=x+50
data=pd.DataFrame(movies)

In [6]:
data.head()

,Title,Year,Runtime (Minutes),Genre,IMDB,Metascore,Description,Director,Stars,Votes,Gross ($)
0,Star Wars: Episode VII - The Force Awakens,2015,138,"Action, Adventure, Sci-Fi",7.8,80,"As a new threat to the galaxy rises, Rey, a de...",J.J. Abrams,"Daisy Ridley,John Boyega,Oscar Isaac,Domhnall ...",958942,"936,662,225"
1,Avengers: Endgame,2019,181,"Action, Adventure, Drama",8.4,78,After the devastating events of Avengers: Infi...,"Anthony Russo, Joe Russo","Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chr...",1220662,"858,373,000"
2,Spider-Man: No Way Home,2021,148,"Action, Adventure, Fantasy",8.2,71,"With Spider-Man's identity now revealed, Peter...",Jon Watts,"Tom Holland,Zendaya,Benedict Cumberbatch,Jacob...",837398,"804,747,988"
3,Avatar,2009,162,"Action, Adventure, Fantasy",7.9,83,A paraplegic Marine dispatched to the moon Pan...,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weaver,M...",1363218,"760,507,625"
4,Top Gun: Maverick,2022,130,"Action, Drama",8.3,78,"After thirty years, Maverick is still pushing ...",Joseph Kosinski,"Tom Cruise,Jennifer Connelly,Miles Teller,Val ...",636146,"718,732,821"


In [7]:
data.describe()

,Title,Year,Runtime (Minutes),Genre,IMDB,Metascore,Description,Director,Stars,Votes,Gross ($)
count,12350,12350,12350,12350,12350,11085,12350,12350,12350,12350,10738
unique,9653,188,161,425,77,98,9998,4126,9952,9308,8198
top,The Lion King,2019,100,"Action, Adventure, Sci-Fi",6.9,68,"As a new threat to the galaxy rises, Rey, a de...","Anthony Russo, Joe Russo","Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chr...",958942,"936,662,225"
freq,96,553,376,850,565,358,48,149,96,48,48


In [8]:
data.isnull().sum()

Title                   0
Year                    0
Runtime (Minutes)       0
Genre                   0
IMDB                    0
Metascore            1265
Description             0
Director                0
Stars                   0
Votes                   0
Gross ($)            1612
dtype: int64

In [9]:
data.dropna(axis=0, inplace=True)

In [10]:
data.isnull().sum()

Title                0
Year                 0
Runtime (Minutes)    0
Genre                0
IMDB                 0
Metascore            0
Description          0
Director             0
Stars                0
Votes                0
Gross ($)            0
dtype: int64

In [11]:
data.duplicated().sum()

2350

In [12]:
data.drop_duplicates(inplace=True)

In [13]:
data.duplicated().sum()

0

In [14]:
data.describe()

,Title,Year,Runtime (Minutes),Genre,IMDB,Metascore,Description,Director,Stars,Votes,Gross ($)
count,7508,7508,7508,7508,7508,7508,7508,7508,7508,7508,7508
unique,7326,164,142,380,77,97,7507,3065,7467,7232,7401
top,Little Women,2006,97,"Comedy, Drama, Romance",6.7,64,A kind-hearted street urchin and a power-hungr...,Clint Eastwood,"William Shatner,Leonard Nimoy,DeForest Kelley,...",9895,"4,360,000"
freq,4,258,222,410,369,179,2,36,6,4,8


In [15]:
data.dtypes

Title                object
Year                 object
Runtime (Minutes)    object
Genre                object
IMDB                 object
Metascore            object
Description          object
Director             object
Stars                object
Votes                object
Gross ($)            object
dtype: object

In [16]:
data['Year']=data['Year'].str.replace('\D','').astype(int)

/tmp/ipykernel_32/1510548062.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Year']=data['Year'].str.replace('\D','').astype(int)


In [17]:
data[['Runtime (Minutes)','Votes']]=data[['Runtime (Minutes)','Votes']].astype(int)
data[['IMDB','Metascore']]=data[['IMDB','Metascore']].astype(float)

In [18]:
data['Gross ($)']=data['Gross ($)'].str.replace(',','').astype('int')

In [19]:
data.dtypes

Title                 object
Year                   int64
Runtime (Minutes)      int64
Genre                 object
IMDB                 float64
Metascore            float64
Description           object
Director              object
Stars                 object
Votes                  int64
Gross ($)              int64
dtype: object

In [20]:
data.to_csv('/kaggle/working/movie_data.csv', index=False)